## Semana 3: 26-30 diciembre + Enero 2
- Revisar modelos contexto legal en español de huggingface
- Implementación de Godel
- Modelos GPT 
- Cap 6 y 7 del libro

# Modelos de Generación de Texto
<font size="4">
Revisión de modelos de generación de texto en la plataforma de hugginface, "los modelos más populares para la tarea de generar texto son los basados en modelos GPT, dado que estos modelos no requieren etiquetas previas en su entrenamiento. Además, los modelos GPT son capaces de generar gran variedad de texto, desde codigo a historias completas." (https://huggingface.co/tasks/text-generation). 
Ejemplos de modelos de generación de texto son _BertGeneration, DialoGPT, Longformer_. Sus diferencias se basan en cambios en los parámetros, bases de datos con los cuales se entrenan y enfoque en atención global o local. 


# GODEL
<font size="4">
GODEL es un modelo de dialogo pre-entrenado desarrollado por Microsoft. Frente a modelos anteriores, presenta la ventaja de una fase de 'grounded pre-training' con la cual se equipa para desarrollar conversaciones que requieren información externa a la conversación actual (bases de datos o ducumentos). 
Utiliza un transformer seq-to-seq para generar respuestas dado un historial de dialogo y un ambiente. El contexto $S$ y el ambiente $E$ son concatenados como una secuencia que será la entrada del modelo.

![Imagen](https://www.marktechpost.com/wp-content/uploads/2022/06/GODEL-1024x533.png)

El modelo se pre-entrena en 3 fases:

1. Pre-entrenamiento lingüístico usando documentos web públicos para proporcionar la capacidad básica para la generación de texto. 
2. Pre-entrenamiento de diálogo sobre datos de diálogo público para mejorar el manejo de los modelos del comportamiento conversacional general.
3. Pre-entrenamiento del contexto para permitir la generación de respuestas según el contexto.

Formato de los datos:

- Context: The context from session beginning to current turn.
- Knowledge: External or environment state represented in plain text.
- Reponse: The target agent respose. It can be a template, an api call or natural language



![Imagen](https://winbuzzer.com/wp-content/uploads/2022/06/GODEL-Language-Model-Chart-Showing-Conversation-Flow.jpg.webp)

_Imagen tomada del artículo [Microsoft’s GODEL Language Model Becomes Open Source](https://winbuzzer.com/2022/06/24/microsofts-godel-language-model-becomes-open-source-xcxwbn/)_

### Ejemplo de aplicación modelo GODEL
<font size="4">


In [2]:
!pip install transformers gradio -q

In [5]:
import gradio as gr

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

In [1]:
preset_examples = [
    ('Instruction: given a dialog context, you need to response empathically.',
     '', 'Does money buy happiness?', 'Chitchat'),
    ('Instruction: given a dialog context, you need to response empathically.',
     '', 'What is the goal of life?', 'Chitchat'),
    ('Instruction: given a dialog context, you need to response empathically.',
     '', 'What is the most interesing thing about our universe?', 'Chitchat'),
     ('Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.', 
     '''Scooby-Doo is the eponymous character and protagonist of the animated television franchise of the same name, created in 1969 by the American animation company Hanna-Barbera.[1] He is a male Great Dane and lifelong companion of amateur detective Shaggy Rogers, with whom he shares many personality traits. He features a mix of both canine and human behaviors (reminiscent of other talking animals in Hanna-Barbera's series), and is treated by his friends more or less as an equal. Scooby often speaks in a rhotacized way, substituting the first letters of many words with the letter 'r'. His catchphrase is "Scooby-Dooby-Doo!"
     ''',
     'What kind of animal is scooby from scooby doo?', 'Conversational Question Answering'
     ),
     ('Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.', 
     '''Subject: faa demos 
    Dan: PM Team, Attached are some general ideas and issues around developing new demos for our new target markets. Please review and provide feedback. Also, please provide links where we can learn more about various FAA applications. Thanx, Dan. 
    Alex: Dan, Thanks for putting the high level descriptions together. My questions are: *Is it practical to do an EAI demo given the inherent complexity of application integration? ... * Should we delay looking at Outlook for now?... *What do you think that timelines are developing these demos? ... Alex 
    Dan: Alex, Thanks for the feedback, please see my comments below:
     ''',
     'what does Dan ask PM team to do?', 'Conversational Question Answering'
     ),
     ('Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.', 
     '''Carlos Alcaraz, at just 19, completed an improbable journey on Sunday in Flushing Meadows as he defeated No. 5 Casper Ruud to win the 2022 US Open. Alcaraz came away with a 6-4, 2-6, 7-6, 6-2 win over Ruud to win his first career Grand Slam title.
     
     In doing so, Alcaraz became the second-youngest player to win a men's US Open title at 19 years, 129 days old, only trailing Pete Sampras. In addition, Alcaraz is the seventh youngest male or female to ever win a Grand Slam tournament. With the Grand Slam victory, Alcaraz becomes the No. 1 ranked player in the world. Additionally, the 19-year-old budding star is also the youngest player to ever be ranked as the world's No. 1 player.
     ''',
     'who won the 2022 US Open? EOS Carlos Alcaraz EOS how old?', 'Conversational Question Answering'
     ),
     (
        'Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.',
        '''Over-the-counter medications such as ibuprofen (Advil, Motrin IB, others), acetaminophen (Tylenol, others) and aspirin.
        ''',
        'I have a headache, what should I do?', "Grounded Response Generation"
     ),
     (
        'Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.',
        '''The best Stardew Valley mods PCGamesN_0 / About SMAPI
        ''',
        'My favorite game is stardew valley. stardew valley is very fun.', "Grounded Response Generation"
     ),
     (
        'Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.',
        '''Wong Kar-wai BBS (born 17 July 1958) is a Hong Kong film director, screenwriter, and producer. His films are characterised by nonlinear narratives, atmospheric music, and vivid cinematography involving bold, saturated colours. A pivotal figure of Hong Kong cinema, Wong is considered a contemporary auteur, and ranks third on Sight & Sound's 2002 poll of the greatest filmmakers of modern times.[note 1] His films frequently appear on best-of lists domestically and internationally.
        ''',
        'My favorite director is wrong kar wai. i think in modern cinema there is no other director is is making the medium as cool', "Grounded Response Generation"
     )
]


/Users/natalia/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 2.36k/2.36k [00:00<00:00, 511kB/s]
Downloading: 100%|██████████| 2.42M/2.42M [00:00<00:00, 3.72MB/s]
Downloading: 100%|██████████| 37.0/37.0 [00:00<00:00, 14.4kB/s]
Downloading: 100%|██████████| 2.20k/2.20k [00:00<00:00, 637kB/s]
Downloading: 100%|██████████| 1.49k/1.49k [00:00<00:00, 368kB/s]
Downloading:  63%|██████▎   | 561M/892M [00:26<00:39, 8.35MB/s] 

KeyboardInterrupt: 

Downloading:  63%|██████▎   | 562M/892M [00:46<00:39, 8.35MB/s]